In [1]:
from CircleNet.system import Simulation
from CircleNet.shape import circle,Timer
from CircleNet.shanghai_platform import MatchingPlatform, Driver, Passenger
from CircleNet.shanghai_platform_extension import extract_result_data
import numpy as np
import time


#put a dictionnary of parameters
#see where is the loss of time
#built a memory of who is first

In [2]:
#automatic functions

def get_id():
    return "{0}{7:03}{3:02}{4:02}{5:02}".format(*time.localtime())

class Write_csv():
    def __init__(self,name,keys_to_save):
        self.file_name=name
        self.the_string="{"+"},{".join(keys_to_save)+"}\n"
        with open(self.file_name , "w") as file :
            file.write(",".join(keys_to_save)+"\n")#write the first line
            assert len(keys_to_save)>0, "nothing to save"
    def write(self,**values):
        with open(self.file_name , "a") as file :
            file.write(self.the_string.format(**values))
 

In [3]:
#simulation parameters

#MOVING
N_driver_list= [2000,3000,4000]#,5000,6000,7000,8000,9000,10000]
N_passenger_list= [2000,3000,4000]#,5000,6000,7000,8000,9000,10000]





#FIXED
speed= 25 #km/h
R= 25 #km
end= 3 #h
#transformation in m, s
speed/=3.6
R*=1000
end*=3600


#in second :
#Drivers:
first_watching_before_first_departure= 5 * 60
window_size_of_departure= 15 * 60
time_elasticity= 5 * 60
fuel_cost= 0.5
watching_repetition_average= 60 # -> random
watching_repetition_variance= 10
time_perception_average= 1 / 300 # = 24 * 50/100 / 3600 #50% of average income, in second   -> random
time_perception_variance= 1 / 3000 #10% percent

#Passenger
publishing_advance= 20 * 60

#TYPE OF SIMULATION
def benefits(origin,destination,network):
    """Shanghai pricing"""
    distance=network.travel_distance(origin,destination)
    if distance < 3000: #3n first km fixprice
        return 11#in RMB
    if distance < 20000:#until 20km at 1.5RMB / km
        return 6.5 + distance * 1.5 # 11 + (distance - 3) * 1.5
    return 8.5 + distance # 28.5 + (distance - 20) *1   # 28.5 = 3 + (20 - 3) * 1.5

def get_log_normal(average,variance):
    sigma=np.log(1+(variance/average)**2)
    mu=np.log(average)-sigma**2/2
    sigma=np.sqrt(sigma)
    def out():
        return np.random.lognormal(mu, sigma)
    return out
watching_repetition=get_log_normal(watching_repetition_average,watching_repetition_variance)
time_perception=get_log_normal(time_perception_average,time_perception_variance)


#agents generator
def SimpleDriver(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    w=(t+first_watching_before_first_departure,t+first_watching_before_first_departure+window_size_of_departure)
    A=w[1]+simulation.network.travel_time(O,D)+time_elasticity
    return Driver(first_watching_time=t,
                  repetition_time=watching_repetition(),
                  departure_window=w,
                  position=O,destination=D,
                  last_arrival_time=A,
                  fuel_cost=fuel_cost,
                  time_perception=time_perception())
def SimplePassenger(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    return Passenger(publishing_time=t,
                     last_departure_time=t+publishing_advance,
                     position=O,destination=D)

In [4]:
#execution

to_save=['id_sim', 'nb_passenger','nb_driver','execution_time', 'nb_match','driver_efficiency', 'passenger_efficiency',  
         'average_waiting_time','average_vks', 'total_vks']
result_file=Write_csv("data/first_sims_after_opt.csv",to_save)

for N_driver in N_driver_list:
    for N_passenger in N_passenger_list :
        print("begin of simulation with ",N_driver," drivers and ",N_passenger," passengers")
        #construction
        N=circle(R,speed)
        T=Timer(end)
        simu=Simulation(N,T)
        simu.matchingAlgo=MatchingPlatform(benefits,simu)
        for i in range(N_driver):
            simu.add(SimpleDriver(simu))
        for i in range(N_passenger):
            simu.add(SimplePassenger(simu))
        id_sim=get_id()
        #execution
        simu()
        results=extract_result_data(simu)
        #saving
        result_file.write(**results,id_sim=id_sim)
    


begin of simulation with  2000  drivers and  2000  passengers
begin of simulation with  2000  drivers and  3000  passengers
begin of simulation with  2000  drivers and  4000  passengers
begin of simulation with  3000  drivers and  2000  passengers
begin of simulation with  3000  drivers and  3000  passengers
begin of simulation with  3000  drivers and  4000  passengers
begin of simulation with  4000  drivers and  2000  passengers
begin of simulation with  4000  drivers and  3000  passengers
begin of simulation with  4000  drivers and  4000  passengers
